In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import rankdata
from sklearn.metrics import auc
def compute_recovery_auc(expression_profile, target_genes):
    """
    Compute the AUC for recovery of target genes in a ranked gene expression profile.
    Target genes with 0 expression are not counted as recovered.
    """
    genes, values = zip(*expression_profile.items())
    ranks = rankdata([-v for v in values], method='average')
    ranked_genes = [gene for _, gene in sorted(zip(ranks, genes))]

    recovery_curve = []
    recovered = 0
    target_count = len(target_genes)

    for i, gene in enumerate(ranked_genes, start=1):
        # Only count recovery if the gene has non-zero expression
        if expression_profile[gene] != 0 and gene in target_genes.values:
            recovered += 1
        recovery_curve.append(recovered / target_count)

    x_vals = np.linspace(0, 1, len(recovery_curve))
    y_vals = np.array(recovery_curve)
    auc_value = auc(x_vals, y_vals)
    return auc_value, recovery_curve

In [ ]:
nichenet_paths = ["CD8DEG_IEL_vs_LP_nichenet_1",
                  "CD4DEG_IEL_vs_LP_nichenet_1",
                  "CD8DEG_IEL_vs_LP_nichenet_2",
                  "CD4DEG_IEL_vs_LP_nichenet_2",
                  "CD8DEG_LP_vs_L_nichenet_1",
                  "CD4DEG_LP_vs_L_nichenet_1",
                  "CD8DEG_LP_vs_L_nichenet_2",
                  "CD4DEG_LP_vs_L_nichenet_2"]

ligand_to_report = pd.DataFrame(index = np.arange(0,30), columns=nichenet_paths)
# curves = pd.DataFrame(index=adata.obs_names, columns=nichenet_paths)
for npath in nichenet_paths:
    target_genes_df = pd.read_csv(npath + '.csv', sep=" ").sort_values(by = "auroc", ascending= False)
    target_genes = target_genes_df[target_genes_df['auroc']>0.5]["test_ligand"].iloc[0:min(30, len(target_genes_df[target_genes_df['auroc']>0.5]["test_ligand"]))]
    ligand_to_report[npath].iloc[0:min(30, len(target_genes_df[target_genes_df['auroc']>0.5]["test_ligand"]))] = target_genes.values
    
ligand_to_report.to_csv('top30_ligand_to_report.csv')
# curves.to_csv('curves.csv')

In [ ]:
%matplotlib inline
nichenet_paths = ["CD8DEG_IEL_vs_LP_nichenet_1",
                  "CD4DEG_IEL_vs_LP_nichenet_1",
                  "CD8DEG_IEL_vs_LP_nichenet_2",
                  "CD4DEG_IEL_vs_LP_nichenet_2",
                  "CD8DEG_LP_vs_L_nichenet_1",
                  "CD4DEG_LP_vs_L_nichenet_1",
                  "CD8DEG_LP_vs_L_nichenet_2",
                  "CD4DEG_LP_vs_L_nichenet_2"]
alternative_names = ['CD8 IEL (to LP)','CD4 IEL (to LP)','CD8 LPL (to IEL)','CD4 LPL (to IEL)','CD8 LPL (to L)','CD4 LPL (to L)','CD8 L (to LP)','CD4 L (to LP)']
plt.rcParams.update({'font.size': 15, 'font.weight': 'heavy','axes.linewidth':2})
#plt.rcParams.update(plt.rcParamsDefault)
kwargs = {'cmap': my_cmap}

for ii, npath in enumerate(nichenet_paths):
    plot_df = pd.read_csv(npath + '.csv', sep=" ", index_col= "test_ligand").sort_values(by = "auroc", ascending= False)["auroc"]
    plot_df = plot_df[plot_df>0.5][0:min(30,plot_df.shape[0])]
    plot_df.index.name = 'influence ' +  alternative_names[ii]
    plot_df.name = 'Activity'
    clustergrid = sns.clustermap(plot_df[0:min(15,plot_df.shape[0])],figsize = (3,6),linewidths=2,
               linecolor = 'k',dendrogram_ratio = 0.1, colors_ratio = 0.5, cbar_pos= None,
               row_cluster = False,col_cluster = False,#cbar_kws={'label': 'NicheNet ligand activity (AUROC)','location':"left"},
               **kwargs)
    # for a in clustergrid.ax_col_dendrogram.collections:
    #     a.set_linewidth(2)
    #     clustergrid.ax_cbar.set_position([-10, 0, -0.3, 0.3])

    axs = clustergrid.fig.get_axes()
    for j in range(0,len(axs)): 
        axs[j].set_xlabel('')

    ax = clustergrid.ax_heatmap

    # Extract data values and annotate each cell
    for i in range(plot_df[0:15].shape[0]):  # Iterate over rows
        for j in range(1):  # Since there's only one column
            value = plot_df[0:15].iloc[i]  # Get value
            ax.text(j+0.5, i+0.5, f'{value:.4f}', ha='center', va='center', color='black', fontsize=12, fontweight='bold')



    plt.savefig(alternative_names[ii]+'ligand_vis.png',dpi = 300,bbox_inches='tight')

## Calculate enrichment (auc)

### Obtain the averaged profile - liver

In [ ]:
# load liver data from GEO - just go download it!
liver_data = pd.read_csv('D:/Trapecar/deconvolute/GSE115469_Data.csv', index_col = 0).T
annotation_liver = pd.read_csv('D:/Trapecar/deconvolute/GSE115469_CellClusterType.txt', sep = '\t')

In [ ]:
liver_data_log = np.log(liver_data + 1) #they didn't log transform it so I'll do it
liver_data_log['cell type']= annotation_liver['CellType'].values

In [ ]:
liver_count = liver_data_log.groupby(['cell type']).mean().unstack().unstack().T
liver_count = liver_count.iloc[:,list((liver_count == 0).all(axis = 0) == 0)]
liver_count_nonT = liver_count.iloc[0:-3,:] # the last 3 rows are T cells. We use our own T cells

In [ ]:
# genes expressed in less than 1% of the cells of a cell type is not expressed
liver_data_bi = liver_data > 0 
liver_data_bi['cell type']= annotation_liver['CellType'].values
celltype_count = liver_data_bi.groupby('cell type').count()*0.01
liver_data_bi_ex = liver_data_bi.groupby('cell type').sum()
liver_data_bi_ex[liver_data_bi_ex>=celltype_count] = -1
# liver_data_bi_ex[liver_data_bi_ex>=10] = -1
liver_data_bi_delete = liver_data_bi_ex>=0

In [ ]:
liver_count_nonT[liver_data_bi_delete] = 0

In [ ]:
liver_count_nonT.to_csv('liver_count_nonT.csv')

### Obtain the averaged profile - gut

In [ ]:
adata_public = sc.read_h5ad('D:/Trapecar/1_Healthy_Pan-GI_atlas_all_lineages_20241119.h5ad')

#### epi

In [ ]:
adata_public_colon_epithelium = adata_public[(adata_public.obs['organ_groups'] == 'Large_intestine') & (adata_public.obs['tissue_fraction'].isin([ 'Epithelium']))]
gut_celltype = adata_public_colon_epithelium.obs['level_2_annot'].value_counts().index[adata_public_colon_epithelium.obs['level_2_annot'].value_counts()>=30]
gut_celltype

In [ ]:
adata_public_colon_epithelium_nonT = adata_public_colon_epithelium[~adata_public_colon_epithelium.obs['level_2_annot'].isin(['Conventional_CD4',
                                                                                                                     'Conventional_CD8',
                                                                                                                     'Cycling_T/NK',
                                                                                                                     'Treg',
                                                                                                                     'Unconventional_T/ILC',
                                                                                                                     'Cycling_T/NK','Glia',
                                                                                                                     'Pericyte',
                                                                                                                     'Neuron_progenitor',
                                                                                                                     'Mature_B',
                                                                                                                     'Cycling_T/NK',
                                                                                                                       'Fibroblast',
                                                                                                                     ]),:]
# not include T cells and not include those whose should not be in the epi

In [ ]:
gut_celltype = adata_public_colon_epithelium_nonT.obs['level_3_annot'].value_counts().index[adata_public_colon_epithelium_nonT.obs['level_3_annot'].value_counts()>=30]
adata_public_colon_epithelium_nonT = adata_public_colon_epithelium_nonT[adata_public_colon_epithelium_nonT.obs['level_3_annot'].isin(gut_celltype),:]
adata_public_colon_epithelium_nonT_df = adata_public_colon_epithelium_nonT.to_df()
adata_public_colon_epithelium_nonT_df['celltype'] = adata_public_colon_epithelium_nonT.obs['level_3_annot']
celltype_profile = adata_public_colon_epithelium_nonT_df.groupby('celltype').mean()

In [ ]:
gut_data_bi = adata_public_colon_epithelium_nonT.to_df() > 0 
gut_data_bi['celltype']= adata_public_colon_epithelium_nonT.obs['level_2_annot']
celltype_count = gut_data_bi.groupby('celltype').count()*0.01
gut_data_bi_ex = gut_data_bi.groupby('celltype').sum()
gut_data_bi_ex[gut_data_bi_ex>=celltype_count] = -1
# gut_data_bi_ex[gut_data_bi_ex>=10] = -1
gut_data_bi_delete = gut_data_bi_ex>=0
gut_data_bi_delete

In [ ]:
celltype_profile[gut_data_bi_delete] = 0

In [ ]:
celltype_profile.to_csv('epithelial_celltype_profile.csv')

#### LP

In [ ]:
adata_public_colon_mucosa= adata_public[(adata_public.obs['organ_groups'] == 'Large_intestine') & (adata_public.obs['tissue_fraction'].isin([ 'Epithelium', 'Lamina_propria','Mucosa']))]
adata_public_colon_epithelium_nonT = adata_public_colon_mucosa[~adata_public_colon_mucosa.obs['level_2_annot'].isin(['Conventional_CD4',
                                                                                                                     'Conventional_CD8',
                                                                                                                     'Cycling_T/NK',
                                                                                                                     'Treg',
                                                                                                                     'Unconventional_T/ILC',
                                                                                                                     'Cycling_T/NK','Glia','Pericyte','Neuron_progenitor',
                                                                                                                     ]),:]

gut_celltype = adata_public_colon_epithelium_nonT.obs['level_2_annot'].value_counts().index[adata_public_colon_epithelium_nonT.obs['level_2_annot'].value_counts()>=30]
adata_public_colon_epithelium_nonT = adata_public_colon_epithelium_nonT[adata_public_colon_epithelium_nonT.obs['level_2_annot'].isin(gut_celltype),:]adata_public_colon_epithelium_nonT_df = adata_public_colon_epithelium_nonT.to_df()
adata_public_colon_epithelium_nonT_df['celltype'] = adata_public_colon_epithelium_nonT.obs['level_2_annot']
adata_public_colon_epithelium_nonT_df = adata_public_colon_epithelium_nonT_df.loc[~adata_public_colon_epithelium_nonT.obs['level_3_annot'].isin(['Villus_fibroblast_F3','Oral_mucosa_fibroblast','Enterocyte']),:]
celltype_profile = adata_public_colon_epithelium_nonT_df.groupby('celltype').mean()
gut_data_bi = adata_public_colon_epithelium_nonT.to_df() > 0 
gut_data_bi['celltype']= adata_public_colon_epithelium_nonT.obs['level_2_annot']
gut_data_bi = gut_data_bi.loc[~adata_public_colon_epithelium_nonT.obs['level_3_annot'].isin(['Villus_fibroblast_F3','Oral_mucosa_fibroblast','Enterocyte']),:]

celltype_count = gut_data_bi.groupby('celltype').count()*0.01
gut_data_bi_ex = gut_data_bi.groupby('celltype').sum()
gut_data_bi_ex[gut_data_bi_ex>=celltype_count] = -1
gut_data_bi_delete = gut_data_bi_ex>=0
celltype_profile[gut_data_bi_delete] = 0
celltype_profile.to_csv('gut_celltype_profile.csv')

### Visualize
Also add our T cell profiles in

In [ ]:
# ligand_to_report = pd.read_csv('top30_ligand_to_report.csv',index_col = 0)
liver_count_nonT = pd.read_csv('liver_count_nonT.csv',index_col = 0)
epi_celltype_profile = pd.read_csv('epithelial_celltype_profile.csv',index_col = 0)
gut_celltype_profile = pd.read_csv('gut_celltype_profile.csv',index_col = 0)

#### liver

In [ ]:
liver_T_df = adata_all[adata_all.obs['tissue'] == 'L',:].to_df()
liver_T_df['celltype'] = adata_all[adata_all.obs['tissue'] == 'L',:].obs['celltype']

In [ ]:
liver_T_count = liver_T_df.groupby('celltype').mean()

In [ ]:
liver_data_bi = adata_all[adata_all.obs['tissue'] == 'L',:].to_df() > 0 
liver_data_bi['cell type']=  adata_all[adata_all.obs['tissue'] == 'L',:].obs['celltype']
celltype_count = liver_data_bi.groupby('cell type').count()*0.1
liver_data_bi_ex = liver_data_bi.groupby('cell type').sum()
liver_data_bi_ex[liver_data_bi_ex>=celltype_count] = -1
# liver_data_bi_ex[liver_data_bi_ex>=10] = -1
liver_data_bi_delete = liver_data_bi_ex>=0

In [ ]:
liver_T_count[liver_data_bi_delete] = 0

In [ ]:
total_liver_df = pd.concat([liver_count_nonT,liver_T_count]).fillna(0)

In [ ]:
nichenet_paths = ["CD8DEG_LP_vs_L_nichenet_2",
                  "CD4DEG_LP_vs_L_nichenet_2"]

liver_enrichment = pd.DataFrame(index=total_liver_df.index, columns=nichenet_paths)
curves = pd.DataFrame(index=total_liver_df.index, columns=nichenet_paths)
for npath in nichenet_paths:
    target_genes = ligand_to_report[npath].dropna()
    for i in range(total_liver_df.shape[0]):
        test_profile = pd.Series(total_liver_df.iloc[i, :], index=total_liver_df.columns).to_dict()
        # print(test_profile)
        auc_value, recovery_curve = compute_recovery_auc(test_profile, target_genes)
        curves[npath][i] = recovery_curve
        liver_enrichment[npath][i] = auc_value

liver_enrichment.to_csv('liver_L_enrichment.csv')

In [ ]:
keep_curves =liver_enrichment.sort_values(by= 'CD8DEG_LP_vs_L_nichenet_2', ascending= False).index[np.r_[0:3, -3:-1]]

In [ ]:
for i in keep_curves:#enrichment.index: #keep_curves:
    for j in [0]:#range(0, curves.shape[1]):
        current_curve = curves.loc[i,:][j]
        x_vals = np.arange(1, len(current_curve) + 1) / len(current_curve)
        y_vals = np.array(current_curve)

        plt.plot(x_vals,y_vals, label = i)
        plt.legend(bbox_to_anchor= [1.6, 1], loc = 'upper right')


In [ ]:
%matplotlib inline
liver_enrichment = liver_enrichment.astype('float').iloc[0:-1,:] #skip the ly49 treg, a very small population
liver_enrichment.columns = ['CD8 L (to LP)','CD4 L (to LP)']
plt.rcParams.update({'font.size': 15, 'font.weight': 'heavy','axes.linewidth':2})
#plt.rcParams.update(plt.rcParamsDefault)
kwargs = {'cmap': 'viridis'}
clustergrid = sns.clustermap(liver_enrichment,figsize = (2,12),linewidths=2,standard_scale=1,
               linecolor = 'k',dendrogram_ratio = 0.1, colors_ratio = 0.5,
               row_cluster = True,col_cluster = False,cbar_kws={'label': 'Normalized AUC Across Subsets','location':"left"},
               **kwargs)
for a in clustergrid.ax_row_dendrogram.collections:
        a.set_linewidth(2)
clustergrid.ax_cbar.set_position([0, 0.3, 0.03, 0.3])

axs = clustergrid.fig.get_axes()
for j in range(0,len(axs)): 
    axs[j].set_xlabel('')

plt.savefig('liver_ligand_auc.png',dpi = 300,bbox_inches='tight')

In [ ]:
%matplotlib inline
alternative_names = ['CD8 L (to LP)','CD4 L (to LP)']
plt.rcParams.update({'font.size': 15, 'font.weight': 'heavy','axes.linewidth':2})
#plt.rcParams.update(plt.rcParamsDefault)
kwargs = {'cmap': 'viridis'}

for i, col in enumerate(ligand_to_report.columns[-2:]):
    heatmap_liver_df = liver_count_nonT.reindex(columns = ligand_to_report[col].dropna(), fill_value=0)
    heatmap_liver_df_norm = (heatmap_liver_df-heatmap_liver_df.min())/(heatmap_liver_df.max()-heatmap_liver_df.min()+0.000001)
    clustergrid = sns.clustermap(heatmap_liver_df_norm,figsize = (12,7),linewidths=2,
                   linecolor = 'k',dendrogram_ratio = 0.05, colors_ratio = 0.5,vmax = 1, vmin = 0,
                   row_cluster = True,col_cluster = False,cbar_kws={'label': 'Normalized expression','location':"left"},
                   **kwargs)
    for a in clustergrid.ax_row_dendrogram.collections:
            a.set_linewidth(2)
    clustergrid.ax_cbar.set_position([-0.05, 0.5, 0.03, 0.3])

    axs = clustergrid.fig.get_axes()
    for j in range(0,len(axs)): 
        axs[j].set_xlabel('')
    clustergrid.fig.suptitle(alternative_names[i], x= 0.4, y = 0.97, ha='center')
    plt.savefig(alternative_names[i]+'_nonT_heat.png',dpi = 300,bbox_inches='tight')
    
    
    heatmap_liver_df = liver_T_count.reindex(columns = ligand_to_report[col].dropna(), fill_value=0)
    heatmap_liver_df_norm = (heatmap_liver_df-heatmap_liver_df.min())/(heatmap_liver_df.max()-heatmap_liver_df.min()+0.000001)
    clustergrid = sns.clustermap(heatmap_liver_df_norm,figsize = (12,5),linewidths=2,
                   linecolor = 'k',dendrogram_ratio = 0.05, colors_ratio = 0.5,vmax = 1, vmin = 0,
                   row_cluster = True,col_cluster = False,cbar_kws={'label': 'Normalized expression','location':"left"},
                   **kwargs)
    for a in clustergrid.ax_row_dendrogram.collections:
            a.set_linewidth(2)
    clustergrid.ax_cbar.set_position([-0.05, 0.5, 0.03, 0.3])

    axs = clustergrid.fig.get_axes()
    for j in range(0,len(axs)): 
        axs[j].set_xlabel('')
    clustergrid.fig.suptitle(alternative_names[i], x= 0.4, y = 0.97, ha='center')
    plt.savefig(alternative_names[i]+'_T_heat.png',dpi = 300,bbox_inches='tight')

#### Colon

##### Epi

In [ ]:
gut_T_df = adata_all[adata_all.obs['tissue'] == 'IEL',:].to_df()
gut_T_df['celltype'] = adata_all[adata_all.obs['tissue'] == 'IEL',:].obs['celltype']

In [ ]:
gut_T_count = gut_T_df.groupby('celltype').mean()

In [ ]:
gut_data_bi = adata_all[adata_all.obs['tissue'] == 'IEL',:].to_df() > 0 
gut_data_bi['celltype']= adata_all[adata_all.obs['tissue'] == 'IEL',:].obs['celltype']
celltype_count = gut_data_bi.groupby('celltype').count()*0.1
gut_data_bi_ex = gut_data_bi.groupby('celltype').sum()
gut_data_bi_ex[gut_data_bi_ex>=celltype_count] = -1
# gut_data_bi_ex[gut_data_bi_ex>=10] = -1
gut_data_bi_delete = gut_data_bi_ex>=0
gut_data_bi_delete

,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,A3GALT2,A4GALT,A4GNT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,hsa-mir-423
celltype,,,,,,,,,,,,,,,,,,,,,
TCRab CD4 Mobile TRM,False,False,True,True,True,True,True,True,True,True,...,False,True,False,False,False,True,False,False,False,True
TCRab CD4 TRM,False,False,True,True,True,True,True,True,True,True,...,False,True,False,False,False,True,False,False,False,True
TCRab CD4 FOXP3+ Treg,False,False,True,True,True,True,True,True,True,True,...,False,False,False,False,False,True,False,False,False,True
TCRab CD8ab TRM,False,False,True,True,True,True,True,True,True,True,...,False,True,False,False,False,True,False,False,False,True
TCRgd TRM,False,False,True,True,True,True,True,True,False,True,...,False,True,False,False,False,True,False,False,False,True
Unconventional Lymphocytes,False,False,True,True,True,True,True,True,True,True,...,False,True,False,False,False,True,False,False,False,True


In [ ]:
gut_T_count[gut_data_bi_delete] = 0

In [ ]:
total_gut_df = pd.concat([epi_celltype_profile,gut_T_count]).fillna(0)

In [ ]:
nichenet_paths = ["CD8DEG_IEL_vs_LP_nichenet_1",
                  "CD4DEG_IEL_vs_LP_nichenet_1",]

enrichment = pd.DataFrame(index=total_gut_df.index, columns=nichenet_paths)
curves = pd.DataFrame(index=total_gut_df.index, columns=nichenet_paths)
for npath in nichenet_paths:
    target_genes = ligand_to_report[npath].dropna()
    for i in range(total_gut_df.shape[0]):
        test_profile = pd.Series(total_gut_df.iloc[i, :], index=total_gut_df.columns).to_dict()
        auc_value, recovery_curve = compute_recovery_auc(test_profile, target_genes)
        curves[npath][i] = recovery_curve
        enrichment[npath][i] = auc_value

columns_vis = ['CD8 IEL (to LP)','CD4 IEL (to LP)']

enrichment = enrichment.astype('float')
enrichment.columns = columns_vis
curves.columns = columns_vis
enrichment.to_csv('epi_enrichment.csv')
# curves.to_csv('curves.csv')

In [ ]:
keep_curves =enrichment.sort_values(by= 'CD8 IEL (to LP)', ascending= False).index[np.r_[0:5, -6:-1]]

In [ ]:
for i in keep_curves:#enrichment.index: #keep_curves:
    for j in [0]:#range(0, curves.shape[1]):
        current_curve = curves.loc[i,:][j]
        x_vals = np.arange(1, len(current_curve) + 1) / len(current_curve)
        y_vals = np.array(current_curve)

        plt.plot(x_vals,y_vals, label = i)
        plt.legend(bbox_to_anchor= [1.6, 1], loc = 'upper right')


In [ ]:
%matplotlib inline
plt.rcParams.update({'font.size': 15, 'font.weight': 'heavy','axes.linewidth':2})
#plt.rcParams.update(plt.rcParamsDefault)
kwargs = {'cmap': 'coolwarm'}
enrichment_norm = (enrichment-enrichment.min())/(enrichment.max()-enrichment.min()+0.000001)
clustergrid = sns.clustermap(enrichment_norm,figsize = (2,10),linewidths=2,vmin = 0, vmax = 1,
               linecolor = 'k',dendrogram_ratio = 0.2, colors_ratio = 0.5,
               row_cluster = True,col_cluster = False,cbar_kws={'label': 'Normalized AUC Across Subsets','location':"left"},
               **kwargs)
for a in clustergrid.ax_row_dendrogram.collections:
        a.set_linewidth(2)
clustergrid.ax_cbar.set_position([0, 0.5, 0.03, 0.3])

axs = clustergrid.fig.get_axes()
for j in range(0,len(axs)): 
    axs[j].set_xlabel('')

plt.savefig('epi_ligand_auc.png',dpi = 300,bbox_inches='tight')

In [ ]:
%matplotlib inline
alternative_names = ['CD8 IEL (to LP)','CD4 IEL (to LP)']
plt.rcParams.update({'font.size': 15, 'font.weight': 'heavy','axes.linewidth':2})
#plt.rcParams.update(plt.rcParamsDefault)
kwargs = {'cmap': 'coolwarm'}

for i, col in enumerate(ligand_to_report.columns[0:2]):
    heatmap_gut_df = gut_T_count.reindex(columns = ligand_to_report[col].dropna(), fill_value=0)
    heatmap_gut_df_norm = (heatmap_gut_df-heatmap_gut_df.min())/(heatmap_gut_df.max()-heatmap_gut_df.min()+0.000001)
    clustergrid = sns.clustermap(heatmap_gut_df_norm,figsize = (12,4),linewidths=2,
                   linecolor = 'k',dendrogram_ratio = 0.1, colors_ratio = 0.5,vmin = 0, vmax = 1,
                   row_cluster = True,col_cluster = False,cbar_kws={'label': 'Normalized expression','location':"left"},
                   **kwargs)
    for a in clustergrid.ax_row_dendrogram.collections:
            a.set_linewidth(2)
    clustergrid.ax_cbar.set_position([-0.03, 0.5, 0.03, 0.3])

    axs = clustergrid.fig.get_axes()
    for j in range(0,len(axs)): 
        axs[j].set_xlabel('')
    clustergrid.fig.suptitle(alternative_names[i], x= 0.4, y = 0.95, ha='center')
    plt.savefig(alternative_names[i]+'_T_heat.png',dpi = 300,bbox_inches='tight')
    
    heatmap_gut_df = epi_celltype_profile.reindex(columns = ligand_to_report[col].dropna(), fill_value=0)
    heatmap_gut_df_norm = (heatmap_gut_df-heatmap_gut_df.min())/(heatmap_gut_df.max()-heatmap_gut_df.min()+0.000001)
    clustergrid = sns.clustermap(heatmap_gut_df_norm,figsize = (12,5),linewidths=2,
                   linecolor = 'k',dendrogram_ratio = 0.1, colors_ratio = 0.5,vmin = 0, vmax = 1,
                   row_cluster = True,col_cluster = False,cbar_kws={'label': 'Normalized expression','location':"left"},
                   **kwargs)
    for a in clustergrid.ax_row_dendrogram.collections:
            a.set_linewidth(2)
    clustergrid.ax_cbar.set_position([-0.03, 0.5, 0.03, 0.3])

    axs = clustergrid.fig.get_axes()
    for j in range(0,len(axs)): 
        axs[j].set_xlabel('')
    clustergrid.fig.suptitle(alternative_names[i], x= 0.4, y = 0.95, ha='center')
    plt.savefig(alternative_names[i]+'_nonT_heat.png',dpi = 300,bbox_inches='tight')

##### LP

In [ ]:
gut_T_df = adata_all[adata_all.obs['tissue'] == 'LP',:].to_df()
gut_T_df['celltype'] = adata_all[adata_all.obs['tissue'] == 'LP',:].obs['celltype']

In [ ]:
gut_T_count = gut_T_df.groupby('celltype').mean()

In [ ]:
gut_data_bi = adata_all[adata_all.obs['tissue'] == 'LP',:].to_df() > 0 
gut_data_bi['celltype']= adata_all[adata_all.obs['tissue'] == 'LP',:].obs['celltype']
celltype_count = gut_data_bi.groupby('celltype').count()*0.01
gut_data_bi_ex = gut_data_bi.groupby('celltype').sum()
gut_data_bi_ex[gut_data_bi_ex>=celltype_count] = -1
# gut_data_bi_ex[gut_data_bi_ex>=10] = -1
gut_data_bi_delete = gut_data_bi_ex>=0
gut_data_bi_delete

,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,A3GALT2,A4GALT,A4GNT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,hsa-mir-423
celltype,,,,,,,,,,,,,,,,,,,,,
TCRab CD4 Naive/TCM,False,True,True,True,True,True,True,True,True,True,...,False,True,False,False,False,True,False,False,False,True
TCRab CD4 Poised TCM,False,False,True,True,True,True,True,True,True,True,...,False,True,False,False,False,True,False,False,False,True
TCRab CD4 Mobile TRM,False,False,True,True,True,True,True,True,True,True,...,False,True,False,False,False,True,False,False,False,True
TCRab CD4 TRM,False,False,True,True,True,True,True,True,True,True,...,False,True,False,False,False,True,False,False,False,True
TCRab CD4 FOXP3+ Treg,False,False,True,True,True,True,True,True,True,True,...,False,True,False,False,False,True,False,False,False,True
TCRab CD4 Tph,False,False,True,True,True,True,True,True,True,True,...,False,True,False,False,False,True,False,False,False,True
TCRab CD8ab TCM,False,False,True,True,True,True,True,True,True,True,...,False,True,False,False,False,True,False,False,False,True
TCRab CD8ab TEM,False,False,True,True,True,True,True,True,True,True,...,False,True,False,False,False,True,False,False,False,True
TCRab CD8ab TRM,False,False,True,True,True,True,True,True,True,True,...,False,True,False,False,False,True,False,False,False,True


In [ ]:
gut_T_count[gut_data_bi_delete] = 0

In [ ]:
total_gut_df = pd.concat([gut_celltype_profile,gut_T_count]).fillna(0)

In [ ]:
nichenet_paths = ["CD8DEG_IEL_vs_LP_nichenet_2",
                  "CD4DEG_IEL_vs_LP_nichenet_2",
                  "CD8DEG_LP_vs_L_nichenet_1",
                  "CD4DEG_LP_vs_L_nichenet_1"]

enrichment = pd.DataFrame(index=total_gut_df.index, columns=nichenet_paths)
curves = pd.DataFrame(index=total_gut_df.index, columns=nichenet_paths)
for npath in nichenet_paths:
    target_genes = ligand_to_report[npath].dropna()
    for i in range(total_gut_df.shape[0]):
        test_profile = pd.Series(total_gut_df.iloc[i, :], index=total_gut_df.columns).to_dict()
        auc_value, recovery_curve = compute_recovery_auc(test_profile, target_genes)
        curves[npath][i] = recovery_curve
        enrichment[npath][i] = auc_value

columns_vis = ['CD8 LPL (to IEL)','CD4 LPL (to IEL)','CD8 LPL (to L)','CD4 LPL (to L)']

enrichment = enrichment.astype('float')
enrichment.columns = columns_vis
curves.columns = columns_vis
enrichment.to_csv('LP_enrichment.csv')
# curves.to_csv('curves.csv')

In [ ]:
keep_curves =enrichment.sort_values(by= 'CD4 LPL (to IEL)', ascending= False).index[np.r_[0:5, -6:-1]]

In [ ]:
for i in keep_curves:#enrichment.index: #keep_curves:
    for j in [0]:#range(0, curves.shape[1]):
        current_curve = curves.loc[i,:][j]
        x_vals = np.arange(1, len(current_curve) + 1) / len(current_curve)
        y_vals = np.array(current_curve)

        plt.plot(x_vals,y_vals, label = i)
        plt.legend(bbox_to_anchor= [1.95, 1], loc = 'upper right')


In [ ]:
%matplotlib inline
plt.rcParams.update({'font.size': 12, 'font.weight': 'heavy','axes.linewidth':2})
#plt.rcParams.update(plt.rcParamsDefault)
kwargs = {'cmap': 'coolwarm'}
enrichment_norm = (enrichment-enrichment.min())/(enrichment.max()-enrichment.min()+0.000001)
clustergrid = sns.clustermap(enrichment_norm.iloc[:,0:2],figsize = (2,16),linewidths=2,vmin = 0, vmax = 1,
               linecolor = 'k',dendrogram_ratio = 0.2, colors_ratio = 0.5,
               row_cluster = True,col_cluster = False,cbar_kws={'label': 'Normalized AUC Across Subsets','location':"left"},
               **kwargs)
for a in clustergrid.ax_row_dendrogram.collections:
        a.set_linewidth(2)
clustergrid.ax_cbar.set_position([0, 0.3, 0.03, 0.3])

axs = clustergrid.fig.get_axes()
for j in range(0,len(axs)): 
    axs[j].set_xlabel('')

plt.savefig('lp_to_iel_ligand_auc.png',dpi = 300,bbox_inches='tight')

In [ ]:
%matplotlib inline
plt.rcParams.update({'font.size': 12, 'font.weight': 'heavy','axes.linewidth':2})
#plt.rcParams.update(plt.rcParamsDefault)
kwargs = {'cmap': 'coolwarm'}
enrichment_norm = (enrichment-enrichment.min())/(enrichment.max()-enrichment.min()+0.000001)
clustergrid = sns.clustermap(enrichment_norm.iloc[:,2:],figsize = (2,16),linewidths=2,vmin = 0, vmax = 1,
               linecolor = 'k',dendrogram_ratio = 0.2, colors_ratio = 0.5,
               row_cluster = True,col_cluster = False,cbar_kws={'label': 'Normalized AUC Across Subsets','location':"left"},
               **kwargs)
for a in clustergrid.ax_row_dendrogram.collections:
        a.set_linewidth(2)
clustergrid.ax_cbar.set_position([0, 0.3, 0.03, 0.3])

axs = clustergrid.fig.get_axes()
for j in range(0,len(axs)): 
    axs[j].set_xlabel('')

plt.savefig('lp_to_l_ligand_auc.png',dpi = 300,bbox_inches='tight')

In [ ]:
%matplotlib inline
alternative_names = ['CD8 LPL (to IEL)','CD4 LPL (to IEL)','CD8 LPL (to L)','CD4 LPL (to L)']
plt.rcParams.update({'font.size': 15, 'font.weight': 'heavy','axes.linewidth':2})
#plt.rcParams.update(plt.rcParamsDefault)
kwargs = {'cmap': 'coolwarm'}

for i, col in enumerate(ligand_to_report.columns[2:6]):
    heatmap_gut_df = gut_T_count.reindex(columns = ligand_to_report[col].dropna(), fill_value=0)
    heatmap_gut_df_norm = (heatmap_gut_df-heatmap_gut_df.min())/(heatmap_gut_df.max()-heatmap_gut_df.min()+0.000001)
    clustergrid = sns.clustermap(heatmap_gut_df_norm,figsize = (12,7),linewidths=2,
                   linecolor = 'k',dendrogram_ratio = 0.1, colors_ratio = 0.5,vmin = 0, vmax = 1,
                   row_cluster = True,col_cluster = False,cbar_kws={'label': 'Normalized expression','location':"left"},
                   **kwargs)
    for a in clustergrid.ax_row_dendrogram.collections:
            a.set_linewidth(2)
    clustergrid.ax_cbar.set_position([-0.03, 0.5, 0.03, 0.3])

    axs = clustergrid.fig.get_axes()
    for j in range(0,len(axs)): 
        axs[j].set_xlabel('')
    clustergrid.fig.suptitle(alternative_names[i], x= 0.4, y = 0.95, ha='center')
    plt.savefig(alternative_names[i]+'_T_heat.png',dpi = 300,bbox_inches='tight')
    
    heatmap_gut_df = gut_celltype_profile.reindex(columns = ligand_to_report[col].dropna(), fill_value=0)
    heatmap_gut_df_norm = (heatmap_gut_df-heatmap_gut_df.min())/(heatmap_gut_df.max()-heatmap_gut_df.min()+0.000001)
    clustergrid = sns.clustermap(heatmap_gut_df_norm,figsize = (12,9),linewidths=2,
                   linecolor = 'k',dendrogram_ratio = 0.1, colors_ratio = 0.5,vmin = 0, vmax = 1,
                   row_cluster = True,col_cluster = False,cbar_kws={'label': 'Normalized expression','location':"left"},
                   **kwargs)
    for a in clustergrid.ax_row_dendrogram.collections:
            a.set_linewidth(2)
    clustergrid.ax_cbar.set_position([-0.03, 0.5, 0.03, 0.3])

    axs = clustergrid.fig.get_axes()
    for j in range(0,len(axs)): 
        axs[j].set_xlabel('')
    clustergrid.fig.suptitle(alternative_names[i], x= 0.4, y = 0.95, ha='center')
    plt.savefig(alternative_names[i]+'_nonT_heat.png',dpi = 300,bbox_inches='tight')